# 🔍 NEXUS AI - Explainable AI (XAI) for AML Detection

**Objective:** Provide transparent, interpretable explanations for ML predictions ensuring regulatory compliance.

**Why XAI is Critical:**
- ✅ **GDPR Article 22** - Right to explanation
- ✅ **Regulatory Audits** - Justify decisions to FinCEN/FATF
- ✅ **Trust & Transparency** - Help analysts understand WHY
- ✅ **Model Debugging** - Identify biases and errors
- ✅ **Feature Discovery** - Find what matters most

**Contents:**
1. Setup & Data Loading
2. Train Baseline Model
3. SHAP (SHapley Additive exPlanations)
4. LIME (Local Interpretable Model-agnostic Explanations)
5. Feature Importance Analysis
6. Individual Prediction Explanations
7. Counterfactual Explanations
8. Global Model Behavior
9. Compliance Reporting
10. Production Deployment

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# XAI libraries
try:
    import shap
    print('✅ SHAP loaded')
    SHAP_AVAILABLE = True
except ImportError:
    print('❌ SHAP not available - install: pip install shap')
    SHAP_AVAILABLE = False

try:
    import lime
    import lime.lime_tabular
    print('✅ LIME loaded')
    LIME_AVAILABLE = True
except ImportError:
    print('❌ LIME not available - install: pip install lime')
    LIME_AVAILABLE = False

try:
    import xgboost as xgb
    print('✅ XGBoost loaded')
except ImportError:
    print('❌ XGBoost not available')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print('\n🔍 NEXUS AI - Explainable AI Module Initialized')

## 1️⃣ Data Loading & Model Training

Load AML transaction data and train a model to explain.

In [ ]:
# Generate synthetic AML data
n_samples = 2000
n_features = 30

np.random.seed(42)
X = np.random.randn(n_samples, n_features)
y = np.zeros(n_samples)
n_suspicious = int(n_samples * 0.05)
suspicious_indices = np.random.choice(n_samples, n_suspicious, replace=False)
y[suspicious_indices] = 1

# Make suspicious transactions distinguishable
X[suspicious_indices, :10] += 2.5
X[suspicious_indices, 10:20] -= 2.0

# Feature names
feature_names = [
    'amount_log', 'amount_zscore', 'hour_of_day', 'day_of_week', 'is_weekend',
    'velocity_7d', 'velocity_30d', 'frequency_7d', 'frequency_30d', 'frequency_change',
    'cross_border', 'high_risk_country', 'round_amount', 'cash_intensive', 'crypto_related',
    'customer_age_days', 'avg_amount_7d', 'avg_amount_30d', 'std_amount_7d', 'std_amount_30d',
    'betweenness_centrality', 'degree_centrality', 'clustering_coef', 'pagerank',
    'time_since_last', 'burst_indicator', 'structuring_indicator', 'layering_score',
    'sanctions_proximity', 'pep_indicator'
]

df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'📊 Data: {n_samples:,} samples, {n_features} features')
print(f'🚨 Suspicious: {y.sum():.0f} ({y.mean()*100:.1f}%)')
print(f'✅ Train: {len(X_train)}, Test: {len(X_test)}')

In [ ]:
# Train XGBoost model
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=19,  # Class imbalance
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

print('🚀 Training XGBoost...')
model.fit(X_train_scaled, y_train)

# Evaluate
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

print(f'\n📊 Model Performance:')
print(f'   Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print(f'   ROC-AUC: {roc_auc_score(y_test, y_proba):.4f}')
print('\n✅ Model ready for explanation!')

## 2️⃣ SHAP (SHapley Additive exPlanations)

**SHAP** provides game-theory based feature attributions.

**Key Concepts:**
- **Shapley values**: Fair contribution of each feature
- **Additive**: Sum of SHAP values = prediction - baseline
- **Consistent**: More contribution = higher SHAP value
- **Local accuracy**: Accurate individual explanations

In [ ]:
if SHAP_AVAILABLE:
    print('🔍 Computing SHAP values...')
    print('   (This may take 1-2 minutes...)\n')
    
    # Create explainer
    explainer = shap.TreeExplainer(model)
    
    # Compute SHAP values
    shap_values = explainer.shap_values(X_test_scaled)
    
    # Handle binary classification
    if isinstance(shap_values, list):
        shap_values = shap_values[1]  # Positive class
    
    print(f'✅ SHAP values computed: {shap_values.shape}')
    print(f'   Base value (expected output): {explainer.expected_value:.4f}')
    print(f'   Mean |SHAP|: {np.abs(shap_values).mean():.4f}')
else:
    print('⚠️  SHAP not available')
    print('   Install: pip install shap')
    shap_values = None

### 2.1 SHAP Summary Plot - Global Feature Importance

In [ ]:
if SHAP_AVAILABLE and shap_values is not None:
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_names, show=False)
    plt.title('🎯 SHAP Feature Importance Summary', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print('\n📊 How to interpret:')
    print('   • Top features = most important globally')
    print('   • Red = high feature value, Blue = low value')
    print('   • Right = increases suspicion, Left = decreases')
    print('   • Width = distribution of impact')

### 2.2 SHAP Bar Plot - Mean Absolute Impact

In [ ]:
if SHAP_AVAILABLE and shap_values is not None:
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_names, 
                     plot_type='bar', show=False)
    plt.title('📊 Mean Absolute SHAP Values', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Top 10
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    top_10 = np.argsort(mean_abs_shap)[-10:][::-1]
    
    print('\n🏆 TOP 10 FEATURES BY IMPACT:')
    print('=' * 60)
    for i, idx in enumerate(top_10, 1):
        print(f'{i:2d}. {feature_names[idx]:30s} | {mean_abs_shap[idx]:.4f}')

### 2.3 Individual Prediction - Suspicious Transaction Explanation

In [ ]:
if SHAP_AVAILABLE and shap_values is not None:
    # Find suspicious transaction
    susp_idx = np.where(y_test == 1)[0]
    if len(susp_idx) > 0:
        idx = susp_idx[0]
        
        print(f'🔍 Explaining Transaction #{idx}')
        print('=' * 70)
        print(f'True Label: SUSPICIOUS')
        print(f'Predicted: {"SUSPICIOUS" if y_pred[idx]==1 else "NORMAL"}')
        print(f'Probability: {y_proba[idx]:.2%}\n')
        
        # Waterfall
        plt.figure(figsize=(14, 8))
        shap.waterfall_plot(
            shap.Explanation(
                values=shap_values[idx],
                base_values=explainer.expected_value,
                data=X_test_scaled[idx],
                feature_names=feature_names
            ),
            show=False
        )
        plt.title(f'💧 Feature Contributions (Transaction #{idx})', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Top contributors
        contribs = list(zip(feature_names, shap_values[idx], X_test_scaled[idx]))
        contribs.sort(key=lambda x: abs(x[1]), reverse=True)
        
        print('\n📋 TOP 10 CONTRIBUTORS:')
        for i, (name, shap_val, feat_val) in enumerate(contribs[:10], 1):
            dir = '↑' if shap_val > 0 else '↓'
            print(f'{i:2d}. {name:30s} {dir} | SHAP: {shap_val:+.4f} | Val: {feat_val:.3f}')

## 3️⃣ LIME (Local Interpretable Model-agnostic Explanations)

LIME approximates the model locally with an interpretable linear model.

In [ ]:
if LIME_AVAILABLE:
    print('🍋 Creating LIME explainer...\n')
    
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train_scaled,
        feature_names=feature_names,
        class_names=['Normal', 'Suspicious'],
        mode='classification',
        random_state=42
    )
    
    print('✅ LIME explainer ready')
else:
    print('⚠️  LIME not available - install: pip install lime')
    lime_explainer = None

### 3.1 LIME Explanation for Same Transaction

In [ ]:
if LIME_AVAILABLE and lime_explainer and len(susp_idx) > 0:
    print(f'🍋 Generating LIME explanation for Transaction #{idx}...\n')
    
    lime_exp = lime_explainer.explain_instance(
        X_test_scaled[idx],
        model.predict_proba,
        num_features=10,
        top_labels=1
    )
    
    # Visualize
    fig = lime_exp.as_pyplot_figure(label=1)
    plt.title(f'🍋 LIME Explanation (Transaction #{idx})', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Details
    print('\n📋 LIME Feature Weights:')
    print('=' * 70)
    for feature, weight in lime_exp.as_list(label=1):
        dir = '↑' if weight > 0 else '↓'
        print(f'{feature:50s} {dir} | {weight:+.4f}')
    
    print(f'\n🎯 LIME prediction: {lime_exp.local_pred[0]:.4f}')
    print(f'🎯 True prediction: {y_proba[idx]:.4f}')
    print(f'✅ Difference: {abs(lime_exp.local_pred[0] - y_proba[idx]):.4f}')

### 2.2 SHAP Bar Plot - Mean Absolute Impact

In [ ]:
if SHAP_AVAILABLE and shap_values is not None:
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test_scaled, feature_names=feature_names, 
                     plot_type='bar', show=False)
    plt.title('📊 Mean Absolute SHAP Values', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Top 10
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    top_10 = np.argsort(mean_abs_shap)[-10:][::-1]
    
    print('\n🏆 TOP 10 FEATURES BY IMPACT:')
    print('=' * 60)
    for i, idx in enumerate(top_10, 1):
        print(f'{i:2d}. {feature_names[idx]:30s} | {mean_abs_shap[idx]:.4f}')

### 2.3 Individual Prediction - Suspicious Transaction Explanation

In [ ]:
if SHAP_AVAILABLE and shap_values is not None:
    # Find suspicious transaction
    susp_idx = np.where(y_test == 1)[0]
    if len(susp_idx) > 0:
        idx = susp_idx[0]
        
        print(f'🔍 Explaining Transaction #{idx}')
        print('=' * 70)
        print(f'True Label: SUSPICIOUS')
        print(f'Predicted: {"SUSPICIOUS" if y_pred[idx]==1 else "NORMAL"}')
        print(f'Probability: {y_proba[idx]:.2%}\n')
        
        # Waterfall
        plt.figure(figsize=(14, 8))
        shap.waterfall_plot(
            shap.Explanation(
                values=shap_values[idx],
                base_values=explainer.expected_value,
                data=X_test_scaled[idx],
                feature_names=feature_names
            ),
            show=False
        )
        plt.title(f'💧 Feature Contributions (Transaction #{idx})', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Top contributors
        contribs = list(zip(feature_names, shap_values[idx], X_test_scaled[idx]))
        contribs.sort(key=lambda x: abs(x[1]), reverse=True)
        
        print('\n📋 TOP 10 CONTRIBUTORS:')
        for i, (name, shap_val, feat_val) in enumerate(contribs[:10], 1):
            dir = '↑' if shap_val > 0 else '↓'
            print(f'{i:2d}. {name:30s} {dir} | SHAP: {shap_val:+.4f} | Val: {feat_val:.3f}')

## 3️⃣ LIME (Local Interpretable Model-agnostic Explanations)

LIME approximates the model locally with an interpretable linear model.

In [ ]:
if LIME_AVAILABLE:
    print('🍋 Creating LIME explainer...\n')
    
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(
        X_train_scaled,
        feature_names=feature_names,
        class_names=['Normal', 'Suspicious'],
        mode='classification',
        random_state=42
    )
    
    print('✅ LIME explainer ready')
else:
    print('⚠️  LIME not available - install: pip install lime')
    lime_explainer = None

### 3.1 LIME Explanation for Same Transaction

In [ ]:
if LIME_AVAILABLE and lime_explainer and len(susp_idx) > 0:
    print(f'🍋 Generating LIME explanation for Transaction #{idx}...\n')
    
    lime_exp = lime_explainer.explain_instance(
        X_test_scaled[idx],
        model.predict_proba,
        num_features=10,
        top_labels=1
    )
    
    # Visualize
    fig = lime_exp.as_pyplot_figure(label=1)
    plt.title(f'🍋 LIME Explanation (Transaction #{idx})', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Details
    print('\n📋 LIME Feature Weights:')
    print('=' * 70)
    for feature, weight in lime_exp.as_list(label=1):
        dir = '↑' if weight > 0 else '↓'
        print(f'{feature:50s} {dir} | {weight:+.4f}')
    
    print(f'\n🎯 LIME prediction: {lime_exp.local_pred[0]:.4f}')
    print(f'🎯 True prediction: {y_proba[idx]:.4f}')
    print(f'✅ Difference: {abs(lime_exp.local_pred[0] - y_proba[idx]):.4f}')